<a href="https://www.kaggle.com/code/keerthi4701/sentiment-analysis-using-bow?scriptVersionId=138818908" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip


# Import Libraries

In [2]:
## importing necessery libraries and data
import numpy as np
import pandas as pd
import re

In [3]:
train =pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', sep='\t')
test = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip', sep = '\t')

In [4]:
train.head(3)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


In [5]:
test.head(3)

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."


In [6]:
train.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

# Preprocessing

In [7]:
from nltk.corpus import stopwords

def preprocess(df):
    df['review'].replace( { r"#(\w+)" : '' }, inplace= True, regex = True) #Remove Hashtags
    df['review'].replace( { r"@(\w+)" : '' }, inplace= True, regex = True) #Remove Mention
    df['review'].astype(str).replace( { r"http\S+" : '' }, inplace= True, regex = True) #Remove URL
    df['review'].replace( { r'[^\w\s]' : '' }, inplace= True, regex = True) # remove punctuation
    df['review'] = df['review'].str.lower() # To lower case

    stop = stopwords.words('english')
    df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [8]:
## preprocessing train and test data
preprocess(train)
preprocess(test)

In [9]:
train.review[0]

'stuff going moment mj ive started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mjs feeling towards press also obvious message drugs bad mkaybr br visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice himbr br actual feature film bit finally starts 20 minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pescis character ranted wanted people know supplying drugs etc dunno maybe hates mjs musicbr br lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence

## TrainTestSplit

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.review, train.sentiment, test_size = 0.2, stratify = train.sentiment)

print(X_train.shape[0])
print(X_test.shape[0])

20000
5000


In [11]:
print('review : \n',X_train.values[0])
print('\nsentiment: \n',y_train.values[0])

review : 
 firstthis movie seems bad almost fell trance first time saw itit like bad dreama cosmic borebut gave second chancethen another anotheretci finally got addicted filmdue dreamlike slow pacewonderful natural setsbathed mellow autumn light especially musical scorewhich made 70s progressive rock absolute exquisite folk songs actorsingersongwriter derek lambthe troubadouryou notice song hazel woodsilver trout lady vanishing airheard middle near end filmthere carnal scenes beginning wich allow us appreciate natural charms elizabeth suzukiif movie made repertoire directors like bergmanlars von triers jeanluc goddardcritics would rolled floorraving movie cosmic masterpiecei personally think film one million times superior fellinis cinematic shdefinitely pretentious

sentiment: 
 1


# CountVenctorizer

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train.values)
X_test_cv = cv.transform(X_test.values)

In [13]:
print(type(X_train_cv))
X_train_cv.toarray()[:2]

<class 'scipy.sparse._csr.csr_matrix'>


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
X_train_cv.shape

(20000, 106847)

In [15]:
print(cv.get_feature_names_out()[10000:10050],"\n")
cv.vocabulary_

['become' 'become_more' 'become_should' 'becomea' 'becomebr' 'becomes'
 'becomesbr' 'becomethis' 'becometrust' 'becoming' 'becomingat' 'becouse'
 'becuase' 'becuz' 'bed' 'beda' 'bedand' 'bedazzled' 'bedbr' 'bedchamber'
 'bedded' 'bedder' 'beddid' 'bedding' 'beddoe' 'bedelia' 'bedeviled'
 'bedford' 'bedhopping' 'bedi' 'bedifferent' 'bedincluding' 'bedknobbr'
 'bedknobs' 'bedlam' 'bednob' 'bedonly' 'bedouin'
 'bedpersonbehindyouinthemirror' 'bedpost' 'bedraggled' 'bedridden'
 'bedroom' 'bedroombr' 'bedroomone' 'bedrooms' 'bedrunkorbedamned' 'beds'
 'bedsheet' 'bedsheets'] 



{'firstthis': 35579,
 'movie': 62626,
 'seems': 83519,
 'bad': 8657,
 'almost': 4772,
 'fell': 34573,
 'trance': 96954,
 'first': 35550,
 'time': 95647,
 'saw': 82156,
 'itit': 49616,
 'like': 55040,
 'dreama': 28545,
 'cosmic': 21679,
 'borebut': 12753,
 'gave': 38629,
 'second': 83342,
 'chancethen': 16968,
 'another': 5860,
 'anotheretci': 5864,
 'finally': 35349,
 'got': 40380,
 'addicted': 3251,
 'filmdue': 35103,
 'dreamlike': 28567,
 'slow': 87129,
 'pacewonderful': 69056,
 'natural': 64098,
 'setsbathed': 84412,
 'mellow': 59701,
 'autumn': 8104,
 'light': 54974,
 'especially': 31872,
 'musical': 63434,
 'scorewhich': 82873,
 'made': 57112,
 '70s': 1887,
 'progressive': 74628,
 'rock': 80129,
 'absolute': 2551,
 'exquisite': 33102,
 'folk': 36332,
 'songs': 88125,
 'actorsingersongwriter': 3121,
 'derek': 25434,
 'lambthe': 53328,
 'troubadouryou': 97627,
 'notice': 65907,
 'song': 88110,
 'hazel': 43180,
 'woodsilver': 104936,
 'trout': 97653,
 'lady': 53183,
 'vanishing': 100

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report,accuracy_score

rf_model = RandomForestClassifier()
gb_model = GradientBoostingClassifier()
lr_model = LogisticRegression(max_iter=1000)
model = MultinomialNB()

In [17]:
model.fit(X_train_cv,y_train)
y_pred = model.predict(X_test_cv)
print("Naive Bayes Model\n")
print(classification_report(y_test,y_pred))

Naive Bayes Model

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      2500
           1       0.87      0.85      0.86      2500

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000



In [18]:
rf_model.fit(X_train_cv,y_train)
y_pred = rf_model.predict(X_test_cv)
print("Random Forest Model\n")
print(classification_report(y_test,y_pred))

Random Forest Model

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      2500
           1       0.85      0.86      0.85      2500

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [19]:
gb_model.fit(X_train_cv,y_train)
y_pred = gb_model.predict(X_test_cv)
print("Gradient Boosting Model\n")
print(classification_report(y_test,y_pred))

Gradient Boosting Model

              precision    recall  f1-score   support

           0       0.84      0.73      0.78      2500
           1       0.76      0.86      0.81      2500

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



In [20]:
lr_model.fit(X_train_cv,y_train)
y_pred = lr_model.predict(X_test_cv)
print("Logistic Regression Model\n")
print(classification_report(y_test,y_pred))

Logistic Regression Model

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      2500
           1       0.87      0.90      0.88      2500

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000



In [21]:
print(accuracy_score(y_test,y_pred))

0.8824


In [22]:
## some random reviews taken from rottentomatos ;)

rev = ["McQuarrie puts enough bloody crunch into the action to dispel any suggestions of creeping comic decadence. Top-flight supporting performances help.", 
       "Open the door, don’t open the door…we fail to care and wish the other dimensional demon would put us all out of our misery.",
       "The long-awaited direct sequel to Insidious: Chapter 2 is a tepid supernatural-horror disappointment.", 
       "“Mission: Impossible – Dead Reckoning Part One” is just incredibly fun. It feels half its length and contains enough memorable action sequences for some entire franchises.",
       "The recut American version is truly awful, but a good 75 percent of the awfulness is attributable to Miramax, the film's distributor.",
       "Despite being the equivalent of a walk through a county fair horror house, this is the Insidious installment with the best executed and sustained suspense."]
rev = cv.transform(rev)
lr_model.predict(rev)

array([1, 0, 0, 1, 0, 1])

# Final Submission


In [23]:
test

,id,review
0,12311_10,naturally film whos main themes mortality nost...
1,8348_2,movie disaster within disaster film full great...
2,5828_4,movie kids saw tonight child loved one point k...
3,7186_2,afraid dark left impression several different ...
4,12128_7,accurate depiction small time mob life filmed ...
...,...,...
24995,2155_10,sony pictures classics im looking sonys got ri...
24996,59_10,always felt ms merkerson never gotten role fit...
24997,2531_1,disappointed movie familiar case read mark fuh...
24998,7772_8,opening sequence filled black white shots remi...


In [24]:
test_reviews = cv.transform(test['review'])
predictions = lr_model.predict(test_reviews)
predictions

array([1, 0, 0, ..., 0, 1, 1])

In [25]:
submission = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv")

In [26]:
type(predictions)

numpy.ndarray

In [27]:
submission["sentiment"] = predictions

In [28]:
submission.to_csv("submission.csv",index = False)

In [29]:
## some random reviews taken from rottentomatos ;)

rev = ["Not bad but not really good. average rating. acting was Dumb",
       "Dead Reckoning” is a perfectly competent entry in a perfectly competent franchise that has carried on for a lot longer -- the first one came out in 1996 -- than anyone could have predicted."]
rev = cv.transform(rev)
lr_model.predict(rev)

array([0, 1])